# PyODB Performance Tests


In [1]:
import cProfile
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from time import sleep, time
import sqlite3 as sql

from pyodb import PyODB, PyODBCache
from test.test_models.complex_models import ComplexBasic, ComplexContainer, ComplexMulti
from test.test_models.primitive_models import PrimitiveBasic, PrimitiveContainer

### Test Uid generator performance

In [5]:
from pyodb._util import generate_uid
start = time()
uids = []
for _ in tqdm(range(1_000_000)):
    uids.append(generate_uid())
print(f"Time taken: {time()-start:.3f}ms")
print(uids[:10])

100%|██████████| 1000000/1000000 [00:00<00:00, 1124607.03it/s]

Time taken: 1.037ms
['5MFNtW4fBllbh_cN08KFJATQxUxQJOfY', 'nvOt-bFgVjXIderElxd6fiesghqhnb0h', 'jBIJOTTV_JXd2CWY8Q9-KJjVm9VqzQsq', 'EObyqGRNoAVsAJ4s-aT-6pSFZY7CuTcd', 'vq82Jog_AInpoQyUZsyKLkbHnjQ89A-0', 'ssWfpElBDJDDuwl2il9v0MQt0z8mo8ak', 'B7WeoJYjkY8ceVllG_ZGfHaBlF9HIRzA', 'wjo_edwa-ZaGnf4WriWRGlZCHmHPppdB', 'iEXmvJm81_JwZKFrdB_neo7quj-l0JR3', 'BkmGJIMEG5c0Ta2AmwL9OxM9MBpzgKpf']


### Test save performance

In [6]:
complex_basic = [ComplexBasic() for _ in range(10000)]
complex_multi = [ComplexMulti() for _ in range(1000)]
complex_container = [ComplexContainer() for _ in range(100)]

def test_insert_performance():
    pyodb = PyODB(0)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    pyodb.save_multiple(complex_basic)
    pyodb.save_multiple(complex_multi)
    pyodb.save_multiple(complex_container)
    del pyodb

base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_insert_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_insert_performance()", filepath.as_posix())


### Test load performance

In [7]:
pyodb = PyODB(0)
pyodb.add_type(ComplexBasic)
pyodb.add_type(ComplexContainer)
pyodb.add_type(ComplexMulti)
print("Generating...")
pyodb.save_multiple([ComplexBasic() for _ in range(10000)])
pyodb.save_multiple([ComplexMulti() for _ in range(1000)])
pyodb.save_multiple([ComplexContainer() for _ in range(100)])
print("Generating done!")

def test_load_performance():
    pyodb.select(ComplexBasic).all()
    pyodb.select(ComplexContainer).all()
    pyodb.select(ComplexMulti).all()

base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_load_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_load_performance()", filepath.as_posix())
del pyodb

Generating...
Generating done!


### Test overall performance

In [8]:
def test_performance():
    pyodb = PyODB(0)
    pyodb.add_type(ComplexBasic)
    pyodb.save(ComplexContainer())
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10)):
        pyodb.save_multiple([ComplexBasic() for _ in range(1000)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])

        pyodb.delete(ComplexBasic).gt(random_number = 0).commit()
        pyodb.remove_type(ComplexContainer)
        pyodb.add_type(ComplexContainer)
    del pyodb


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_overall_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_performance()", filepath.as_posix())

100%|██████████| 10/10 [00:21<00:00,  2.14s/it]


In [9]:
def insert_base_data():
    pyodb = PyODB(persistent=True)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10), desc="Inserting base data"):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])


def test_select_performance():
    pyodb = PyODB()
    for _ in tqdm(range(100), desc="Testing PrimitiveBasic"):
        pyodb.select(PrimitiveBasic).all()
    for _ in tqdm(range(100), desc="Testing PrimitiveContainer"):
        pyodb.select(PrimitiveContainer).all()
    for _ in tqdm(range(100), desc="Testing ComplexBasic"):
        pyodb.select(ComplexBasic).all()
    for _ in tqdm(range(100), desc="Testing ComplexContainer"):
        pyodb.select(ComplexContainer).all()


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_select_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
insert_base_data()
cProfile.run("test_select_performance()", filepath.as_posix())

Testing ComplexContainer: 100%|██████████| 100/100 [00:00<00:00, 140.30it/s]


In [11]:
cache = PyODBCache()
cache.pyodb.persistent = True
cache.add_cache("test", lambda x: [ComplexMulti() for _ in range(x)], ComplexMulti, 2)
print(cache.get_data("test", 100))

start = time()
print(cache["test"])
print(time() - start)

sleep(2)
print(cache.get_data("test", 1))
del cache

[<test.test_models.complex_models.ComplexMulti object at 0x7f43eb8aaa10>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86a950>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86b790>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb869090>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb868610>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86af90>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb868190>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb869d10>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86bc50>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb869a90>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86a5d0>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb868b10>, <test.test_models.complex_models.ComplexMulti object at 0x7f43eb86a7d0>, <test.test_models.complex_models.ComplexMulti obje

## PyODB Examples

### Basic Example

In [ ]:
class MyType:
    some_data: list[str]
    some_number: int | None

    def __init__(self, number: int):
        self.some_data = ["Hello", "World"]
        self.some_number = number
    
    def __repr__(self) -> str:
        return f"MyType: {self.some_number}"

# Create PyODB instance
pyodb = PyODB()

# Add type and save some instances
pyodb.add_type(MyType)
pyodb.save(MyType(1))
pyodb.save_multiple([MyType(2), MyType(3), MyType(4), MyType(5)])

# Need data elsewhere
select = pyodb.select(MyType)
## only get instances where some_number > 2
select.gt(some_number = 2)
## Res now contains the members
res = select.all()
print(res)

# The select can also be done in a one-liner
res = pyodb.select(MyType).gt(some_number = 2).all()
print(res)

# Delete the saved entries
deleted = pyodb.delete(MyType).gt(some_number = 2).commit()
print(f"Deleted {deleted} entries")

# Count remaining entries
count = pyodb.select(MyType).count()
print(f"{count} entries remaining")

# Clear the database keeping the table definitions
pyodb.clear()

# Show and then remove the type definition
print(pyodb.known_types)
pyodb.remove_type(MyType)
print(pyodb.known_types)


### DBConn Performance Test

In [ ]:
pyodb.add_type(PrimitiveBasic)
pyodb.save_multiple([PrimitiveBasic() for i in range(1000)])

for i in tqdm(range(1000), desc="Re-creating DBConns"):
    res = sql.connect(
        "./.pyodb/pyodb.db"
    ).execute(
        "SELECT * FROM \"test.test_models.primitive_models.PrimitiveBasic\""
    ).fetchall()

conn = sql.connect("./.pyodb/pyodb.db")
for i in tqdm(range(1000), desc="Using one DBConn"):
    res = conn.execute(
        "SELECT * FROM \"test.test_models.primitive_models.PrimitiveBasic\""
    ).fetchall()
